<a href="https://colab.research.google.com/github/mgholamikn/AdaptPose/blob/main/AdaptPose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AdaptPose: Cross-dataset Adaptation of 3D Human Pose Estimation by Learnable Motion Generator**

Here we provide the following experiments: 

1.   Cross-dataset evaluation on 3DHP: source H3.6M, target: 3DHP
2.   Cross-dataset evaluation on 3DPW: source H3.6M, target: 3DPW
3.   Cross-dataset trainin on 3DHP: source H3.6M, target: 3DHP
4.   Cross-dataset trainin on 3DPW: source H3.6M, target: 3DPW


# **Install Dependencies**

In [1]:
%pip install certifi==2020.12.5
%pip install cffi==1.14.0
%pip install cycler==0.10.0
%pip install kiwisolver==1.3.1
%pip install matplotlib==3.1.3
%pip install mkl-fft==1.3.0
%pip install mkl-random==1.1.1
%pip install mkl-service==2.3.0
%pip install nbconvert==5.6.1
%pip install numpy 
%pip install olefile==0.46
%pip install Pillow 
%pip install protobuf==3.15.6
%pip install pycparser 
%pip install pyparsing==2.4.7
%pip install python-dateutil==2.8.1
%pip install scipy==1.4.1
%pip install six 
%pip install tensorboardX==1.6
%pip install torch
%pip install torchgeometry==0.1.2
%pip install torchvision==0.2.2

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/5e/a0/5f06e1e1d463903cf0c0eebeb751791119ed7a4b3737fdc9a77f1cdfb51f/certifi-2020.12.5-py2.py3-none-any.whl (147 kB)
  Attempting uninstall: certifi
    Found existing installation: certifi 2022.12.7
    Uninstalling certifi-2022.12.7:
      Successfully uninstalled certifi-2022.12.7
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.1/463.1 kB 759.1 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/62/d5/5f610ebe421e85889f2e55e33b7f9a6795bd982198517d912eb1c76e1a53/pycparser-2.21-py2.py3-none-any.whl (118 kB)
  Created wheel for cffi: filename=cffi-1.14.0-cp39-cp39-linux_x86_64.whl size=182060 sha256=e8b474d4469a6585efae5892c8c1348cc58df5a05bbe43ea0f35fd47956c5a66
  Stored in directory: /home/data/yyc/.cache/pip/wheels/8b/fa/38/20cba8ccc332f

In [ ]:
import sys
import torch
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{torch.__version__[0:5:2]}"
])
%pip install pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html

# 下载数据

实验室服务器连不到google，需要手动下载后上传

In [ ]:
#%cd /TO/AdaptPose/ 
!ls
%cd data
## download data
!gdown --id 1_cJ2vG5wxRQUtWavz47XAwaMvU9pQB_X
!gdown --id 1NX9dT1nKX-8t_eZR3vnfrdyOegG7pDBs
!gdown --id 1v3-LDnjlHT8OJi40E7OKDBGY7dWM9P2n
!gdown --id 12voJG5DBFy_2xT3lNdLYA4ZCTYjs7Iyv
!gdown --id 1Y96GlQOEkc2Gx6V4FpvTwCyJjrUX3FYu
!gdown --id 1vL53iJ1mWao3TKK3p52ZX_jdF5d5EBtl
!gdown --id 1EbVV-nzrrQ2KdT_GG4JPwv1A7EOnFka1
## download pretrained models
%cd ..
%cd checkpoint/adaptpose/videopose/gt/3dhp/
!gdown --id 11QJf-B5D1_aqsMR3v-9rrfht4W7ErC4k
%cd ../../../../..
%cd checkpoint/adaptpose/videopose/gt/3dpw/
!gdown --id 1LUi1OQ7vWQ5KXJ9HlSvtveiOEacG1WoS
%cd ../../../../..
%cd checkpoint/adaptpose/videopose/gt/skii/
!gdown --id 1b1_A7SuBdGaPNXzsx2qLOg0_RqIuk6Cx
%cd ../../../../..
%cd checkpoint/pretrain_baseline/videopose/gt/3dhp/
!gdown --id 1SmSXrk_LcpsXBpBEsSp0zq9qIpb4aJBU
%cd ../../../../..

# **Experiments:**

**1. Cross-dataset Evaluation of Pretrained Model on 3DHP dataset**




In [ ]:
!python3  run_evaluate.py --posenet_name 'videopose' --keypoints gt --evaluate  'checkpoint/adaptpose/videopose/gt/3dhp/ckpt_best_dhp_p1.pth.tar' --dataset_target 3dhp --keypoints_target 'gt' --pad 13  --pretrain_path  'checkpoint/pretrain_baseline/videopose/gt/3dhp/ckpt_best.pth.tar'

**2. Cross-dataset Evaluation of Pretrained Model on 3DPW dataset**

In [ ]:
!python3  run_evaluate.py --posenet_name 'videopose' --keypoints gt --evaluate  'checkpoint/adaptpose/videopose/gt/3dpw/ckpt_best_dhp_p1.pth.tar' --dataset_target 3dpw --keypoints_target 'gt' --pad 13  --pretrain_path  'checkpoint/pretrain_baseline/videopose/gt/3dpw/ckpt_best.pth.tar'

**3. Cross-dataset Training for 3DHP**

In [ ]:
!python3 run_adaptpose.py --note poseaug --posenet_name 'videopose' --lr_p 1e-4 --checkpoint './checkpoint/adaptpose' --keypoints gt --keypoints_target gt --dataset_target '3dhp'  --pretrain_path './checkpoint/pretrain_baseline/videopose/gt/3dhp/ckpt_best.pth.tar'  --pad 13 

**2.Cross-dataset Evaluation for 3DPW**

In [ ]:
!python3 run_adaptpose.py --note poseaug --posenet_name 'videopose' --lr_p 1e-4 --checkpoint './checkpoint/adaptpose' --keypoints gt --keypoints_target gt --dataset_target '3dpw'  --pretrain_path './checkpoint/pretrain_baseline/videopose/gt/3dhp/ckpt_best.pth.tar'  --pad 13 